# 통계적 검정력 (Statistical Power)
통계적 검정력을 달성한다는 의미는 아주 단순한 모델보다 성능이 개선된 모델을 개발한다는 의미입니다.  
MNIST 숫자 이미지 분류에서는 10%보다 높은 정확도를 보여주는 모델이 통계적 검정력을 가졌다고 말 할 수 있습니다.  
IMDB 영화 리뷰 분류에서는 50%보다 높은 정확도를 보여주는 모델이 통계적 검정력을 가졌다고 말 할 수 있습니다.  
  
검정력은 우리가 세운 가설이 참일 때 이를 채택할 확률을 말합니다.  
여기에서는 적어도 데이터셋에 있는 클래스별 분포보다 모델의 정확도가 높아야 우리가 세운 가설이 옳다고 말할 수 있습니다.  
우리의 가설은 주어진 입력으로 출력을 예측할 수 있다는 가설입니다.   
유튜브: https://youtu.be/tXkaj5XC7RU

# 드롭아웃(Dropout)
토론토 대회의 제프리 힌튼은 과대적합을 해소하는 드롭아웃을 개발했습니다.  
힌튼은 은행에서 사용하는 부정 방지 메커니즘에서 드롭아웃을 착안했습니다.  
힌튼이 은행에 갔을 때 행원들이 계속 바뀌길래 왜 그런지를 물었습니다.  
자신들도 이유는 모르지만 자주 업무가 바뀐다고 했습니다.  
힌튼은 은행에서 부정 행위를 하려면 직원들 사이의 유대가 필요하기 때문이라고 판단했습니다.  
그래서 딥러닝 모델에서도 뉴런의 부정한 협업을 방지하고 과대적합을 감소시키기 위해 각 샘플에 대해 뉴런의 일부를 무작위하게 제거했습니다. 

In [112]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adagrad

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from matplotlib import pyplot as plt
%matplotlib inline

In [113]:
df = pd.read_csv('data.csv')

df['매수시간(분)'] = pd.to_datetime(df['매수시간'])
df['매수시간(분)'] = df['매수시간(분)'].apply(lambda x : x.time())
df['매수시간(분)'] = df['매수시간(분)'].apply(lambda x : x.hour*60 + x.minute)

df['주문일자'] = pd.to_datetime(df['주문일자'])
df['매수시간(초)'] = pd.to_datetime(df['매수시간'])
df['매수시간(초)'] = df['매수시간(초)'].apply(lambda x : x.time().second)

df['매수시간'] = pd.to_datetime(df['매수시간'])
df['최대거래대금 시간'] = pd.to_datetime(df['최대거래대금 시간'])
df['최대거래대금 시차(초)'] = df['매수시간'] - df['최대거래대금 시간']
df['최대거래대금 시차(초)'] = df['최대거래대금 시차(초)'].apply(lambda x : x.total_seconds())

df['수익률'] = df['수익률'].apply(lambda x : 1 if x > 0 else 0 )

X = df[['매매순서','최대거래대금','직전 거래대금','양봉개수','10이격도','20이격도','60이격도','매수등락률','시가등락률','매수시간(분)','매수시간(초)','분봉전고점','특징주','SEN','강세 토픽', '테마 리포트', '지정예고', '최대거래대금 시차(초)']]
y = df['수익률']

X = X.to_numpy()
y = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [114]:
model = Sequential()
model.add(Dense(8, input_dim=18, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adagrad(lr=0.1)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [115]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 8)                 152       
_________________________________________________________________
dropout_21 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 4)                 36        
_________________________________________________________________
dropout_22 (Dropout)         (None, 4)                 0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 5         
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [116]:
model.fit(X_train, y_train, epochs=160, batch_size=40)

Epoch 1/160
147/147 [==============================] - 1s 6ms/step - loss: 0.7795 - acc: 0.4830
Epoch 2/160
147/147 [==============================] - 0s 61us/step - loss: 0.6724 - acc: 0.5850
Epoch 3/160
147/147 [==============================] - 0s 54us/step - loss: 0.6349 - acc: 0.6327
Epoch 4/160
147/147 [==============================] - 0s 61us/step - loss: 0.6241 - acc: 0.6463
Epoch 5/160
147/147 [==============================] - 0s 41us/step - loss: 0.6056 - acc: 0.6327
Epoch 6/160
147/147 [==============================] - 0s 61us/step - loss: 0.5811 - acc: 0.6395
Epoch 7/160
147/147 [==============================] - 0s 41us/step - loss: 0.6130 - acc: 0.6599
Epoch 8/160
147/147 [==============================] - 0s 61us/step - loss: 0.5775 - acc: 0.6871
Epoch 9/160
147/147 [==============================] - 0s 54us/step - loss: 0.5851 - acc: 0.6871
Epoch 10/160
147/147 [==============================] - 0s 54us/step - loss: 0.5917 - acc: 0.6395
Epoch 11/160
147/147 [========

147/147 [==============================] - 0s 61us/step - loss: 0.4895 - acc: 0.7415
Epoch 85/160
147/147 [==============================] - 0s 54us/step - loss: 0.4287 - acc: 0.7755
Epoch 86/160
147/147 [==============================] - 0s 61us/step - loss: 0.4336 - acc: 0.7959
Epoch 87/160
147/147 [==============================] - 0s 54us/step - loss: 0.3977 - acc: 0.8027
Epoch 88/160
147/147 [==============================] - 0s 54us/step - loss: 0.4278 - acc: 0.8027
Epoch 89/160
147/147 [==============================] - 0s 48us/step - loss: 0.4202 - acc: 0.7891
Epoch 90/160
147/147 [==============================] - 0s 41us/step - loss: 0.4282 - acc: 0.7755
Epoch 91/160
147/147 [==============================] - 0s 41us/step - loss: 0.4618 - acc: 0.7755
Epoch 92/160
147/147 [==============================] - 0s 41us/step - loss: 0.4314 - acc: 0.7891
Epoch 93/160
147/147 [==============================] - 0s 48us/step - loss: 0.4621 - acc: 0.7619
Epoch 94/160
147/147 [===========

In [117]:
_, accuracy = model.evaluate(X_test, y_test)
print(accuracy)

50/50 [==============================] - 0s 6ms/step
0.7199999928474426


In [118]:
y_pred = model.predict(X_test)
confusion_matrix = metrics.confusion_matrix(y_test, np.rint(y_pred))
confusion_matrix

array([[31,  3],
       [11,  5]], dtype=int64)

# 한 쪽 방향으로만 맞추는 아주 단순한 모델의 정확도
34/50=0.68

In [119]:
len(y_test)

50

In [120]:
len(y_test[y_test == 1])

16

In [110]:
len(y_test[y_test == 0])

34

In [111]:
34/50

0.68